In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import kpss, adfuller

# SARIMA

In [ ]:
LOCATION = "Nelson Street"

In [ ]:
cycle_counts_path = "../cycle_counts.csv"
cycle_counts = pd.read_csv(cycle_counts_path, parse_dates=["date"])
cycle_counts = cycle_counts[cycle_counts["location"] == LOCATION]

In [ ]:
fig, ax = plt.subplots()
ax.plot(cycle_counts["date"], cycle_counts["count"], lw=1.5)
ax.set(title=LOCATION, ylabel="Count")
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
fig.tight_layout();

In [ ]:
cycle_counts["date"] = pd.to_datetime(cycle_counts["date"])
cycle_counts = cycle_counts.set_index("date").drop(columns=["location"])
cycle_counts = cycle_counts.resample("D").sum()
cycle_counts = cycle_counts.fillna(0)

## Autocorrelation

### Augmented Dickey Fuller (ADF) Test

- **Null Hypothesis:** Timeseries is non-stationary.
- **Alternate Hypothesis:** Timeseries is stationary.

### Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test

- **Null Hypothesis:** Timeseries is stationary.
- **Alternate Hypothesis:** Timeseries is non-stationary.


**Note,** that both tests assume that the data is non-seasonal. Need other ways of formally testing for stationarity in seasonal data, see for example: https://otexts.com/fpp3/stationarity.html

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True,)

plot_acf(cycle_counts["count"], ax=ax[0])
ax[0].set(xlabel="Lag")

plot_pacf(cycle_counts["count"], ax=ax[1])
ax[1].set(xlabel="Lag")

fig.tight_layout();

### Differencing

#### Seasonal Differencing

In [ ]:
seasonal_diffs = cycle_counts["count"].diff(7).dropna()
seasonal_diffs.plot()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True,)

plot_acf(seasonal_diffs, ax=ax[0])
ax[0].set(xlabel="Lag")

plot_pacf(seasonal_diffs, ax=ax[1])
ax[1].set(xlabel="Lag")

fig.tight_layout();

In [ ]:
results_adf = adfuller(seasonal_diffs)
print(f"ADF Results:\ntest statistic: {results_adf[0]:.2f}\np-value: {results_adf[1]:.2f}")

In [ ]:
results_kpss = kpss(seasonal_diffs)
print(f"KPSS Results:\ntest statistic: {results_kpss[0]:.2f}\np-value: {results_kpss[1]:.2f}")

### SARIMA Model

In [ ]:
model = SARIMAX(
    endog=cycle_counts["count"],
    order=(0, 0, 1),  # (1, 0, 2)
    seasonal_order=(0, 1, 1, 7),  # (0, 1, 1, 7)
)
model = model.fit()
y_hat = model.predict(start=cycle_counts.index[0], end=cycle_counts.index[-1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(cycle_counts["count"], label="Observed")
ax.plot(y_hat, label="Fitted")
ax.set(title=LOCATION, ylabel="Count")
ax.legend()
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
fig.tight_layout();

### Residuals

In [ ]:
resid = cycle_counts["count"] - y_hat

fig, ax = plt.subplots(2, 2, figsize=(12, 5.5))
ax = ax.flatten()

ax[0].hist(np.array(resid), bins=20)
ax[0].set(ylabel="Frequency", xlabel="Residuals")

ax[1].axhline(0, color="black", linestyle="--")
ax[1].scatter(np.array(cycle_counts["count"]), np.array(resid))
ax[1].set(xlabel="Observed", ylabel="Residuals")

ax[2].plot(cycle_counts["count"].index, np.array(resid))
ax[2].set(ylabel="Residuals")
for tick in ax[2].get_xticklabels():
    tick.set_rotation(45)

plot_acf(resid, ax=ax[3])
ax[3].set(title="", xlabel="Lag", ylabel="ACF")

fig.tight_layout();

### Forecasting

In [ ]:
n_test = 21

train = cycle_counts["count"].iloc[: -n_test]
val = cycle_counts["count"].iloc[-n_test:]

model = SARIMAX(
    endog=train,
    order=(1, 0, 2),
    seasonal_order=(0, 1, 1, 7),
)
model = model.fit()

In [ ]:
forecasts = model.get_prediction(
    start=val.index.min(),
    end=val.index.max()
)

predicted_mean = forecasts.predicted_mean.to_frame(name="mean")
conf_int_90 = (
    forecasts.conf_int(alpha=0.10)
    .rename(columns={"lower count": "lower_0.10", "upper count": "upper_0.10"})
)
conf_int_95 = (
    forecasts.conf_int(alpha=0.05)
    .rename(columns={"lower count": "lower_0.05", "upper count": "upper_0.05"})
)
forecasts_df = pd.concat([predicted_mean, conf_int_90, conf_int_95], axis=1)

In [ ]:
fig, ax = plt.subplots()

ax.plot(train.index.values[-50:], train.values[-50:], lw=2.5, color="black")

ax.fill_between(
    forecasts_df.index.values,
    forecasts_df["lower_0.10"].values,
    forecasts_df["upper_0.10"].values,
    alpha=0.2,
    color="blue"
)
ax.fill_between(
    forecasts_df.index.values,
    forecasts_df["lower_0.05"].values,
    forecasts_df["upper_0.05"].values,
    alpha=0.1,
    color="blue"
)
ax.plot(
    forecasts_df.index.values,
    forecasts_df["mean"].values,
    lw=2.5,
    color="blue",
    ls="--"
)

ax.plot(val.index.values, val.values, lw=2.5, color="blue")
ax.set(ylabel="Counts")

fig.tight_layout();